In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

### A little preliminary Data Wrangling

In [4]:
# load in the dataset into a pandas dataframe
diamonds = pd.read_csv('./diamonds.csv')